In [25]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils.vis_utils import plot_model
import os

In [26]:
num_classes = 3 
img_rows, img_cols = 64, 64
batch_size=8

In [27]:
train="E:/Assignment 3/train"
test="E:/Assignment 3/test"

In [28]:
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)
train_generator = train_gen.flow_from_directory(
        train,target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
test_generator = test_gen.flow_from_directory(
        test,target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

Found 2554 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


## Defining the model

In [29]:
model = Sequential()
model.add(Conv2D(32,(3,3), padding='same',input_shape= (img_rows,img_cols,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
activation_19 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 62, 62, 32)        9248      
_________________________________________________________________
activation_20 (Activation)   (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 31, 31, 64)       

In [36]:
from keras.optimizers import Adam,SGD,RMSprop,Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(r"E:\Assignment 3\IMG_CNN_1.h5",
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                         min_delta = 0,
                         patience = 3,
                         verbose = 1,
                         restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = "val_loss",
                            factor = 0.2,
                            patience = 3,
                            verbose = 1,
                            min_delta = 0.0001)

callbacks = [earlystop, checkpoint, reduce_lr]
model.compile(loss = 'categorical_crossentropy',
             optimizer=Adam(learning_rate=0.01),
             metrics = ['accuracy'])
train_samp=2554
test_samp=30
epochs = 20

history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_samp // batch_size,
    epochs= epochs,
    callbacks=callbacks,
    validation_data = test_generator,
    validation_steps = test_samp // batch_size)

Epoch 1/20
319/319 [==============================] - 4s 12ms/step - loss: 1.0842 - accuracy: 0.4195 - val_loss: 1.1362 - val_accuracy: 0.2917

Epoch 00001: val_loss improved from inf to 1.13624, saving model to E:\Assignment 3\IMG_CNN_1.h5
Epoch 2/20
319/319 [==============================] - 4s 12ms/step - loss: 1.0836 - accuracy: 0.4214 - val_loss: 1.0986 - val_accuracy: 0.4545

Epoch 00002: val_loss improved from 1.13624 to 1.09856, saving model to E:\Assignment 3\IMG_CNN_1.h5
Epoch 3/20
319/319 [==============================] - 4s 12ms/step - loss: 1.0844 - accuracy: 0.4179 - val_loss: 1.0672 - val_accuracy: 0.3182

Epoch 00003: val_loss improved from 1.09856 to 1.06723, saving model to E:\Assignment 3\IMG_CNN_1.h5
Epoch 4/20
319/319 [==============================] - 4s 12ms/step - loss: 1.0838 - accuracy: 0.4203 - val_loss: 1.1212 - val_accuracy: 0.2727

Epoch 00004: val_loss did not improve from 1.06723
Epoch 5/20
319/319 [==============================] - 4s 12ms/step - loss:

In [43]:
callbacks = [earlystop, checkpoint, reduce_lr]
model.compile(loss = 'categorical_crossentropy',
             optimizer=SGD(learning_rate=0.1, momentum=0.9),
             metrics = ['accuracy'])

In [44]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_samp // batch_size,
    epochs= epochs,
    callbacks=callbacks,
    validation_data = test_generator,
    validation_steps = test_samp // batch_size)

Epoch 1/20
319/319 [==============================] - 3s 11ms/step - loss: 1.1011 - accuracy: 0.4042 - val_loss: 1.1245 - val_accuracy: 0.3750

Epoch 00001: val_loss did not improve from 1.04165
Epoch 2/20
319/319 [==============================] - 3s 10ms/step - loss: 1.1045 - accuracy: 0.3940 - val_loss: 1.1008 - val_accuracy: 0.4091

Epoch 00002: val_loss did not improve from 1.04165
Epoch 3/20
319/319 [==============================] - 3s 10ms/step - loss: 1.0991 - accuracy: 0.3959 - val_loss: 1.1685 - val_accuracy: 0.3636

Epoch 00003: val_loss did not improve from 1.04165
Epoch 4/20
319/319 [==============================] - 3s 10ms/step - loss: 1.1101 - accuracy: 0.3857 - val_loss: 1.0372 - val_accuracy: 0.3636

Epoch 00004: val_loss improved from 1.04165 to 1.03722, saving model to E:\Assignment 3\IMG_CNN_1.h5
Epoch 5/20
319/319 [==============================] - 3s 10ms/step - loss: 1.1087 - accuracy: 0.3896 - val_loss: 0.9665 - val_accuracy: 0.4167

Epoch 00005: val_loss impr

In [45]:
callbacks = [earlystop, checkpoint, reduce_lr]
model.compile(loss = 'categorical_crossentropy',
             optimizer=RMSprop(learning_rate=0.1),
             metrics = ['accuracy'])

In [46]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_samp // batch_size,
    epochs= epochs,
    callbacks=callbacks,
    validation_data = test_generator,
    validation_steps = test_samp // batch_size)

Epoch 1/20
319/319 [==============================] - 4s 12ms/step - loss: 1.0978 - accuracy: 0.4022 - val_loss: 1.0495 - val_accuracy: 0.3333

Epoch 00001: val_loss did not improve from 0.96649
Epoch 2/20
319/319 [==============================] - 3s 10ms/step - loss: 1.0970 - accuracy: 0.4030 - val_loss: 1.1634 - val_accuracy: 0.3182

Epoch 00002: val_loss did not improve from 0.96649
Epoch 3/20
319/319 [==============================] - 3s 10ms/step - loss: 1.0945 - accuracy: 0.4030 - val_loss: 1.1374 - val_accuracy: 0.3182

Epoch 00003: val_loss did not improve from 0.96649
Epoch 4/20
319/319 [==============================] - 3s 10ms/step - loss: 1.0937 - accuracy: 0.3951 - val_loss: 1.1621 - val_accuracy: 0.3636
Restoring model weights from the end of the best epoch

Epoch 00004: val_loss did not improve from 0.96649

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.020000000298023225.
Epoch 00004: early stopping


In [47]:
callbacks = [earlystop, checkpoint, reduce_lr]
model.compile(loss = 'categorical_crossentropy',
             optimizer=Nadam(learning_rate=0.001),
             metrics = ['accuracy'])

In [48]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_samp // batch_size,
    epochs= epochs,
    callbacks=callbacks,
    validation_data = test_generator,
    validation_steps = test_samp // batch_size)

Epoch 1/20
319/319 [==============================] - 5s 15ms/step - loss: 1.0872 - accuracy: 0.4199 - val_loss: 0.9698 - val_accuracy: 0.3750

Epoch 00001: val_loss did not improve from 0.96649
Epoch 2/20
319/319 [==============================] - 4s 14ms/step - loss: 1.0835 - accuracy: 0.4199 - val_loss: 1.1939 - val_accuracy: 0.3182

Epoch 00002: val_loss did not improve from 0.96649
Epoch 3/20
319/319 [==============================] - 4s 13ms/step - loss: 1.0835 - accuracy: 0.4175 - val_loss: 1.2444 - val_accuracy: 0.1818

Epoch 00003: val_loss did not improve from 0.96649
Epoch 4/20
319/319 [==============================] - 4s 13ms/step - loss: 1.0817 - accuracy: 0.4226 - val_loss: 1.1856 - val_accuracy: 0.4545
Restoring model weights from the end of the best epoch

Epoch 00004: val_loss did not improve from 0.96649

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 00004: early stopping


In [50]:
callbacks = [earlystop, checkpoint, reduce_lr]
model.compile(loss = 'categorical_crossentropy',
             optimizer=SGD(learning_rate=0.1, nesterov=True),
             metrics = ['accuracy'])

In [51]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_samp // batch_size,
    epochs= epochs,
    callbacks=callbacks,
    validation_data = test_generator,
    validation_steps = test_samp // batch_size)

Epoch 1/20
319/319 [==============================] - 3s 11ms/step - loss: 1.0843 - accuracy: 0.4199 - val_loss: 1.1035 - val_accuracy: 0.2917

Epoch 00001: val_loss did not improve from 0.96649
Epoch 2/20
319/319 [==============================] - 3s 10ms/step - loss: 1.0828 - accuracy: 0.4199 - val_loss: 1.1317 - val_accuracy: 0.3182

Epoch 00002: val_loss did not improve from 0.96649
Epoch 3/20
319/319 [==============================] - 3s 10ms/step - loss: 1.0845 - accuracy: 0.4218 - val_loss: 1.0975 - val_accuracy: 0.4091

Epoch 00003: val_loss did not improve from 0.96649
Epoch 4/20
319/319 [==============================] - 3s 10ms/step - loss: 1.0845 - accuracy: 0.4183 - val_loss: 1.2016 - val_accuracy: 0.3182

Epoch 00004: val_loss did not improve from 0.96649
Epoch 5/20
319/319 [==============================] - 3s 10ms/step - loss: 1.0859 - accuracy: 0.4187 - val_loss: 1.0925 - val_accuracy: 0.3333

Epoch 00005: val_loss did not improve from 0.96649
Epoch 6/20
319/319 [=====